In [31]:
import torch
from collections import OrderedDict

In [32]:
clip = torch.load('ViT-L-14-336px.pt').state_dict()

In [35]:
def convert_openclip(ckpt):
    new_ckpt = OrderedDict()

    for k, v in list(ckpt.items()):
        if 'visual' not in k:
            continue
        k = k.replace('visual.', '')
        k = k.replace('transformer.', '')
        new_v = v
        if k.startswith('proj'):
            new_k = 'neck.proj'
            new_ckpt[new_k] = new_v
            continue
        elif k.startswith('conv1'):
            new_k = k.replace('conv1', 'patch_embed.projection')
        elif k.startswith('ln_pre'):
            new_k = k.replace('ln_pre.', 'pre_norm.')
        elif k.startswith('resblocks'):
            new_k = k.replace('resblocks.', 'layers.')
            if 'ln_1' in k:
                new_k = new_k.replace('ln_1', 'ln1')
            elif 'ln_2' in k:
                new_k = new_k.replace('ln_2', 'ln2')
            elif 'in_proj_weight' in k:
                new_k = new_k.replace('in_proj_weight', 'qkv.weight')
            elif 'in_proj_bias' in k:
                new_k = new_k.replace('in_proj_bias', 'qkv.bias')
            if 'out_proj' in k:
                new_k = new_k.replace('out_proj', 'proj')
            elif 'mlp.c_fc' in k:
                new_k = new_k.replace('mlp.c_fc', 'ffn.layers.0.0')
            elif 'mlp.c_proj' in k:
                new_k = new_k.replace('mlp.c_proj', 'ffn.layers.1')
        elif k.startswith('class_embedding'):
            new_v = v[None, None, :]
            new_k = k.replace('class_embedding', 'cls_token')
        elif k.startswith('positional_embedding'):
            new_v = v[None, ...]
            new_k = k.replace('positional_embedding', 'pos_embed')
        elif k.startswith('ln_post'):
            new_k = k.replace('ln_post', 'ln1')
        else:
            new_k = k

        if not new_k.startswith('head'):
            new_k = 'backbone.' + new_k
        new_ckpt[new_k] = new_v
    return new_ckpt

In [36]:
x = convert_openclip(clip)

In [38]:
torch.save(x, 'clip2mmcls_converted.pth')